Предположил, что отправленные из 1 кластера во второй деньги - это то, что получил второй кластер. А fees отправитель платит дополнительно.

In [184]:
import pandas as pd

Прочитаем данные и запишем их в датафреймы

In [185]:
clust = pd.read_csv('address_clust.csv', index_col='address_id')
clust.head()

,cluster_id
address_id,
71413451,1
71411914,1
71410369,1
71410100,1
71410040,1


In [186]:
stats = pd.read_csv('address_stats.csv', index_col='id')
stats.head()

,address_id,transaction_id,received,sent
id,,,,
87134765,46402336,19162323,0.0,1800000.0
87134766,45919235,19162323,0.0,1071052.0
87134767,46529090,19162323,1000000.0,0.0
87134768,46529091,19162323,1821052.0,0.0
87154706,46529090,19166856,0.0,1000000.0


Выведем количество адресов, принадлежащих 1 и 2 кластеру

In [187]:
clust.cluster_id.value_counts()

1    3652
2    3344
Name: cluster_id, dtype: int64

Добавим к таблице stats еще один столбец - номер кластера. Если значения stats.address_id нет в таблице clust, значит адрес принадлежит кластеру 0

In [188]:
full_clusters = stats.address_id.apply(lambda s: clust.cluster_id[s] if s in clust.index else 0)
stats['cluster_id'] = full_clusters
stats.head()

,address_id,transaction_id,received,sent,cluster_id
id,,,,,
87134765,46402336,19162323,0.0,1800000.0,0
87134766,45919235,19162323,0.0,1071052.0,0
87134767,46529090,19162323,1000000.0,0.0,1
87134768,46529091,19162323,1821052.0,0.0,0
87154706,46529090,19166856,0.0,1000000.0,1


Проверка на то, что в каждой транзакции деньги сначала отправляются, а потом получаются. То есть нет такого, что кто-то отправил, другой получил, а после этого в этой же транзакции кто-то еще отправил (возможно, это очевидно, но я в блокчейне не разбирался пока). Данная ячейка не выводит ничего

In [189]:
last_trans = None
rec = False
for index, row in stats.iterrows():
  if last_trans != row.transaction_id:
    rec = False
    last_trans = row.transaction_id
  else:
    if row.received > 0:
      rec = True
    if rec and row.sent > 0:
      print(row.transaction_id)
      break

Проверим еще одно предположение: отправитель в каждой транзакции всегда один. Это оказалось действительно так, потому что ячейка ничего не вывела

In [190]:
last_trans = None
for index, row in stats.iterrows():
  if last_trans != row.transaction_id:
    if row.sent > 0:
      cl_sent = row.cluster_id
    last_trans = row.transaction_id
  else:
    if row.sent > 0 and cl_sent != row.cluster_id:
      print(row.transaction_id)
      break

Если в каждой транзакции деньги сначала кто-то отправляет, а потом кто-то получает, можно для каждой транзакции и каждого кластера, участвующего в ней, посчитать сумму отправленных и полученных денег.

In [191]:
sent = stats.groupby(['transaction_id', 'cluster_id'])['sent'].sum()

In [192]:
received = stats.groupby(['transaction_id', 'cluster_id'])['received'].sum()

Функция получения суммы отправленных денег между двумя кластерами

In [193]:
def get_sent_received(first_cl, second_cl):
  first_to_second = 0
  second_to_first = 0
  # рассмотрим операции внутри каждой транзакции
  for tr in stats.transaction_id.unique():
    # если рассматриваемые кластеры не участвуют в транзакции, пропускаем её
    if first_cl not in received[tr].index or second_cl not in received[tr].index:
      continue

    # если отправляет первый, то добавляем полученное вторым к соответствующей переменной
    if first_cl == sent[tr].idxmax():
      first_to_second += received[tr][second_cl]
    # если отправляет второй, то добавляем полученное первым к соответствующей переменной
    elif second_cl == sent[tr].idxmax():
      second_to_first += received[tr][first_cl]
  
  # переведем сатоши в биткоины
  first_to_second, second_to_first = first_to_second / 1e8, second_to_first / 1e8

  return first_to_second, second_to_first

In [194]:
first, second = get_sent_received(1, 2)
print('Первый кластер отправил во второй: {:.2f} биткоинов,\nвторой кластер отправил в первый {:.2f} биткоинов'.format(first, second))

Первый кластер отправил во второй: 129.24 биткоинов,
второй кластер отправил в первый 135.82 биткоинов


In [195]:
first, second = get_sent_received(0, 1)
print('Нулевой кластер отправил в первый: {:.2f} биткоинов,\nпервый кластер отправил в нулевой {:.2f} биткоинов'.format(first, second))

Нулевой кластер отправил в первый: 297.12 биткоинов,
первый кластер отправил в нулевой 300.55 биткоинов


In [196]:
first, second = get_sent_received(0, 2)
print('Нулевой кластер отправил во второй: {:.2f} биткоинов,\nвторой кластер отправил в нулевой {:.2f} биткоинов'.format(first, second))

Нулевой кластер отправил во второй: 9.74 биткоинов,
второй кластер отправил в нулевой 4.87 биткоинов


In [197]:
def get_fees(cluster):
  fees = 0
  for tr in stats.transaction_id.unique():
    # если кластер не участвует в транзакции или не отправляет деньги, пропускаем транзакцию
    if cluster not in sent[tr].index or sent[tr][cluster] == 0:
      continue

    # добавляем к fees разницу отправленного и полученного в данной транзакции
    fees += sent[tr][cluster] - received[tr].sum()

  # сатоши в биткоины
  fees /= 1e8
  return fees

In [198]:
fees = get_fees(1)
print('Первый кластер потратил на fees: {:.2f} биткоинов'.format(fees))

Первый кластер потратил на fees: 0.67 биткоинов


In [199]:
fees = get_fees(2)
print('Второй кластер потратил на fees: {:.2f} биткоинов'.format(fees))

Второй кластер потратил на fees: 0.75 биткоинов


### Ответы:

Первый кластер отправил во второй: 129.24 биткоинов,
второй кластер отправил в первый 135.82 биткоинов

Нулевой кластер отправил в первый: 297.12 биткоинов,
первый кластер отправил в нулевой 300.55 биткоинов

Нулевой кластер отправил во второй: 9.74 биткоинов,
второй кластер отправил в нулевой 4.87 биткоинов

Первый кластер потратил на fees: 0.67 биткоинов

Второй кластер потратил на fees: 0.75 биткоинов